In [40]:
!pip install  -r ../requirements_pipelines.txt

In [41]:
# Python Imports
import os
import re

# Third party imports
from dotenv import load_dotenv


# Langchain imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Load the environment variables
load_dotenv(override=True)

True

In [42]:
class ProcessData:
    def __init__(self,
                data_directory: str="../data/APEC_ChromaDB_VectorStore",
                history_file: str="../data/processed_files.txt",
                embedding_model: str="text-embedding-3-large",
                ):
        
        # List to store the documents temporarily
        self.documents = []

        # History file
        self.history_file = history_file

        # Define the Database to store the embeddings
        self.data_directory = data_directory
        # Define the initial text splitter by "\n\n" line break
        self.text_splitter = CharacterTextSplitter(
                                                    separator="\n",  # Use two line breaks as separator
                                                    chunk_size=500,  # Don't limit the chunk size
                                                    chunk_overlap=50,  # No overlap between chunks
                                                    length_function=len,
                                                    is_separator_regex=False  # Indicate that the separator is not a regular expression
                                                    )

        

        # Embedding Model
        self.embedding_openai = OpenAIEmbeddings(model=embedding_model)

    def process_pdf(self, filepath: str):
        """
        This Method processes a PDF file and put the documents in to the self.documents list
        :param filepath: The path to the PDF file
        :return: None
        """
        print("Processing the file: ", filepath)
        # Verify if the file was already processed
        # if not self.verify_file_was_already_processed(filepath):
        #     return 'Document already processed before'
        
        # Load the file
        loader = UnstructuredPDFLoader(filepath)
        data = loader.load()

        # Split the text into chunks
        documents_pdf = self.text_splitter.split_documents(data)
        print("Number of documents: ", len(documents_pdf))
        # Save the processed data
        self.documents.extend(documents_pdf)

        # Verify the number of documents
        number_documents = len(self.documents)

        # If there are many docuemnts save and free the memory
        if number_documents > 50:

            # Save the data
            status_save = self.save_procceced_data_into_vector_store(self.documents)

            # Verify if the process was successful
            if status_save == 'Success':
                self.documents = []
                print("The data was already proccesed and saved")

        # Save the file in the history
        with open(self.history_file, 'a') as file:
            file.write(filepath + "\n")

        print(self.documents)
        return 'Success'
    
    def save_procceced_data_into_vector_store(self, documents_to_save: list):
        """
        This method saves the processed data in to a list
        :param data: The data to be saved
        :return: None
        """
        # Create a Chroma Vector Store
        vectorstore_chroma = Chroma.from_documents(
                                                    documents=documents_to_save,
                                                    embedding=self.embedding_openai,
                                                    persist_directory=self.data_directory
                                                    )
        
        # Persist the data
        vectorstore_chroma.persist()

        return 'Success'
    
    def verify_file_was_already_processed(self, filepath: str):
        """
        This method verifies if the file was already processed
        :param filepath: The path to the file
        :return: None
        """
        # Read the history file
        with open(self.history_file, 'r') as file:
            history_files = file.read().splitlines()

        # Verify if the file is in the history
        if filepath in history_files:
            return True
        
        return False

Processing the data to a Vector Store Database

In [44]:
# Define the base path to process the data
base_path = '/mnt/i'

# Create an Object to process the data
process_data = ProcessData()

# Iterate over all files in a directory
for dirpath, dirnames, filenames in os.walk(base_path):

    # Explore the files
    for filename in filenames:
        
        # Extract the extension
        file_extension = os.path.splitext(filename)[1].lower()

        if file_extension.strip() == '.pdf':
            
            # Process the PDF file
            process_data.process_pdf(os.path.join(dirpath, filename))

Processing the file:  /mnt/i/DX_Promote_Enhanced_Car_Wash_for_AX12_and_Anthem_UX_Rev_02_1_.pdf


ModuleNotFoundError: No module named 'cv2.typing'; 'cv2' is not a package